## Import necessary packages to collect and process data from the MTA Developers Site

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import inspect

### 1. Collect one file of MTA data to a database

In [2]:
mta_df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_210904.txt')

Change this file to .csv form

In [3]:
mta_df.to_csv('mta_data_210904.csv', header = False, index = False)

### 2. Create the Database

- In terminal, I used the code `sqlite3 mta.db` to open `sqlite3` and create a new database
- Additionally, `sqlite3 mta.db` is the command I use, in general, to open the database

### 3. Create the table

- In terminal, I creaeted a table within the databaseto to store all the MTA data, called: **mta_data**
- Now in `sqlite>`, I created my table with the following SQL statement: 

 ```sql
 CREATE TABLE mta_data (
	CA TEXT,
	UNIT TEXT,
	SCP TEXT,
	STATION TEXT,
	LINENAME TEXT,
	DIVISION TEXT,
	DATE TEXT,
	TIME TEXT,
	DESC TEXT,
	ENTRIES INTEGER,
	EXITS INTEGER,
	PRIMARY KEY (CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS)
	);
```

- I then added one row of data to test out the `INSERT INTO` functionality with the following SQL statement:

```sql
INSERT INTO mta_data (
	CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS)
	VALUES ('A002','R051','02-00-00','59 ST','NQR456W','BMT','03/06/2021','03:00:00','REGULAR',0007540642,0002572027);
```

- Then to avoid issues with duplicate/incorrect rows, I deleted the recently added row with the following command:

```sql
DELETE FROM mta_data;
```

### 4. Insert an entire file into the table

- Within `sqlite3`, I ran the following command line to insert my entire .csv into the database

```sql
.mode csv
.import mta_data_210904.csv mta_data
```

`.mode csv` changes the `sqlite3` mode to csv, while the `.import mta_data_210904.csv mta_data` imports the csv file into the pre existing table

- Then to avoid issues with duplicate/incorrect rows, I deleted the recently added row with the following command:

```sql
DELETE FROM mta_data;
```

### 5. Add multiple files of MTA data to the database using Python

I used the following script to loop through MTA url's to collect the .txt files of my desired weeks of turnstile data

In [4]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt" #url with formatting to fill in the number of the particular week
    dfs = []
    for week in week_dates: #for each week
        file_url = url.format(week) #format the url so it grabs that week
        dfs.append(pd.read_csv(file_url)) #append to our empty list
    return pd.concat(dfs) #combine files for each week
        
week_dates = [210605, 210612, 210619, 210626, 210703, 210710, 210717, 210724, 210731, 210807, 210814, 210821, 210828, 210904]
turnstiles_df = get_data(week_dates)

Once all the data is concatinated into one dataframe, `turnstiles_df`, I then exported it to a csv to evenntually be accessed via `SQLAlchemy` in Python

In [5]:
turnstiles_df.to_csv('mta_data_all.csv', header = True, index = False)

# set 'header = True' to keep column names from MTA's web site, however, we made already created the column names in sqlite3, 
# therefore header = False is more appropriate as we would end up with two header rows

### 6. Getting the data into the Database and Table


- Back in terminal, within sqlite3 I ran the following command line to insert my new `mta_data_all.csv` into the `mta.db` 

```sql
.mode csv
.import mta_data_all.csv mta_data
```

### 7. Light Exploratory Data Analysis (EDA) in SQL

I used the `create_engine` function within `SQLAlchemy` to begin EDA on the `mta_data`

In [6]:
engine = create_engine("sqlite:///mta.db")
inspect(engine).get_table_names()

['mta_data']

Lastly, I used `pd.read_sql_table()` to express the ability to read an SQL database table into a Pandas DataFrame and assign it to a variable

In [7]:
df = pd.read_sql('mta_data', engine) # note: this process can take a significant amount of time

In [8]:
# First Operation (LIKE, WHERE used)

pd.read_sql('SELECT * FROM mta_data WHERE station LIKE "t%" LIMIT 10;', engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/29/2021,00:00:00,REGULAR,9986327,12523278
1,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/29/2021,04:00:00,REGULAR,9986348,12523302
2,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/29/2021,08:00:00,REGULAR,9986378,12523372
3,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/29/2021,12:00:00,REGULAR,9986463,12523563
4,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/29/2021,16:00:00,REGULAR,9986596,12523830
5,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/29/2021,20:00:00,REGULAR,9986780,12524144
6,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/30/2021,00:00:00,REGULAR,9986882,12524274
7,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/30/2021,04:00:00,REGULAR,9986899,12524303
8,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/30/2021,08:00:00,REGULAR,9986916,12524333
9,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,05/30/2021,12:00:00,REGULAR,9986997,12524467


In [9]:
# Second Operation (AND, WHERE used) 

pd.read_sql('SELECT * FROM mta_data WHERE station = "ORCHARD BEACH" AND desc = "REGULAR" LIMIT 10;', engine)

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/29/2021,00:00:00,REGULAR,401648,24781
1,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/29/2021,04:00:00,REGULAR,401648,24781
2,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/29/2021,08:00:00,REGULAR,401650,24783
3,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/29/2021,12:00:00,REGULAR,401650,24783
4,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/29/2021,16:00:00,REGULAR,401650,24786
5,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/29/2021,20:00:00,REGULAR,401650,24786
6,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/30/2021,00:00:00,REGULAR,401650,24786
7,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/30/2021,04:00:00,REGULAR,401650,24786
8,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/30/2021,08:00:00,REGULAR,401650,24786
9,OB01,R459,00-00-00,ORCHARD BEACH,6,IND,05/30/2021,12:00:00,REGULAR,401650,24787


In [10]:
# Third Operation (HAVING used)

pd.read_sql('SELECT station FROM mta_data GROUP BY date HAVING COUNT(unit) > 5;', engine)

,STATION
0,59 ST
1,59 ST
2,59 ST
3,59 ST
4,59 ST
...,...
93,59 ST
94,59 ST
95,59 ST
96,59 ST


In [11]:
# Fourth Operation (DISTINCT used)

pd.read_sql('SELECT DISTINCT station FROM mta_data;', engine)

,STATION
0,59 ST
1,5 AV/59 ST
2,57 ST-7 AV
3,49 ST
4,TIMES SQ-42 ST
...,...
374,EASTCHSTER/DYRE
375,ST. GEORGE
376,TOMPKINSVILLE
377,RIT-MANHATTAN
